# Install & setup LaminDB

## Installation

```{include} installation.md

```

## Sign up & log in

```{include} signup-login.md

```

### Sign up

```{code}

lamin signup testuser1@lamin.ai
```

This will generate a password and cache both email and password in your `~/.lamin` directory.

📧 You will also receive a confirmation email with a link to choose your user handle and complete the sign-up.

:::{dropdown} Which errors might I see during sign up?

If you try to sign up again, you'll see an error message:

```

User already exists! Please login instead: `lamin login`.

```

If you did not confirm the signup email, you'll see:

```

RuntimeError: It seems you already signed up with this email. Please click on the link in the confirmation email that you should have received from lamin.ai.

```

Depending on timing, you might see a `429 Too Many Requests` error.
:::

### Log in

You can log in with either email or handle:

```
lamin login testuser1@lamin.ai
lamin login testuser1
```

If you don't have a cached password in your environment, you need to pass it:

```{code}
lamin login <email> --password <password>
```

## Help

Access help on the CLI:
```
lamin -h
lamin init -h
```

## Docker

If you'd like a docker container, here is a way: [github.com/laminlabs/lamindb-docker](https://github.com/laminlabs/lamindb-docker).

## Init an instance

### Parameters

- `storage`: a default storage location (e.g. `s3://my-bucket`, `gs://my-bucket`, `./my-data-dir`)
- `name` (optional): a name for the instance (e.g., `my-project-assets`, `analyses-team-x`, `ml-experiments-y`)
- `db` (optional): a SQL database URI (defaults to SQLite)
- `schema` (optional): comma-separated schema names
    - available plug ins are listed [here](/lnschema-bionty)
    - [contact us](https://lamin.ai/contact) to learn about customized enterprise plug ins

### Examples

#### Local storage + SQLite

If you are only interested in tracking files and their transformations, init your local SQLite instance via:

```
lamin init --storage ./mydata
```

Mount the Bionty schema module:

```
lamin init --storage mydata --schema bionty
```

#### S3 + SQLite

```
lamin init --storage s3://<bucket_name> --schema bionty,lamin1
```

#### GCP + Postgres

```
lamin init --storage gs://<bucket_name> --db postgresql://<user>:<pwd>@<hostname>:<port>/<dbname> --schema bionty,lamin1
```


## Load an instance

Load your own instance:
```
lamin load <instance_name>
````

Load somebody else's instance:
```
lamin load <account_handle/instance_name>
```

## Access settings

Now, let's look at a specific example:

In [ ]:
!lamin init --storage mydata --schema bionty

Print settings:

In [ ]:
!lamin info

Settings persist in `~/.lamin/` and can also be accessed via {class}`lamindb.setup.settings`.

In [ ]:
import lamindb as ln

In [ ]:
ln.setup.settings.user

In [ ]:
ln.setup.settings.instance

```{note}

- The user who creates an instance is its owner. Ownership can be transferred in the hub.
- Advanced users could also consider the Python setup API: {mod}`lamindb.setup`.

```

## Update default storage

It's easiest to see and update default storage in the Python API using {attr}`~lamindb.dev.Settings.storage`:

```python
import lamindb as ln
ln.settings.storage  # set via ln.settings.storage = "s3://other-bucket"
#> s3://default-bucket
```

You can also change it using the CLI via

```
lamin set --storage s3://other-bucket
```

## Close an instance

Loading an instance means loading an environment for managing your datasets.

When loading a new instance, you automatically _close_ the previously loaded old instance.

If you want to close the instance without loading a new instance, use `lamin close`

## Migrate an instance

If you are an admin and you haven't set up automated deployments of migrations, you can use two commands to create and deploy migrations:

- `lamin migrate create`
- `lamin migrate deploy`

Unless you manage a custom plugin schema, you'll _never_ need to create a migration.

You'll receive a logged warning when deploying a migration is advisable.

:::{dropdown} How does this warning look like?

Here is an example:

```
% lamin load testdb
🔶 

Your database is not up to date with your installed Python library.

The database misses the following migrations:
[<Migration lnschema_core.0014_rename_ref_field_featureset_registry>, <Migration lnschema_core.0015_file_initial_version_file_version>]

Only if you are an admin and manage migrations manually, deploy them to the database:
lamin migrate deploy

Otherwise, downgrade your Python library to match the database!

✅ loaded instance: testuser1/testdb
```
:::

### Create a migration

You need to have the schema package installed locally:
```
git clone https://github.com/my-org/lnschema-custom
cd lnschema-custom
pip install -e .
```

Edit the registries in your schema.

Then, call

```
lamin migrate create
```

to create the migration script.

When you're happy, commit them to your GitHub repo, and ideally make a new release.

To deploy the migration call `lamin migrate deploy`.

```{note}

The `lamin` migration commands are a wrapper around Django's migration manager.

```

## Delete an instance

This works as follows. It won't delete your data, just the metadata managed by LaminDB:

In [ ]:
!lamin delete --force mydata